In [1]:
import pandas as pd

# For checking URL and HTTP errors
from urllib.error import URLError, HTTPError

A Mutual Fund Name can be the same but the NAV Name has two types of Funds
1. Regular: Growth & IDCW (Income Distribution cum Capital Withdrawal)
2. Direct: Growth & IDCW (Income Distribution cum Capital Withdrawal)

In [2]:
# --- GLOBAL VARIABLES ---

# AMFI Site URL that holds a CSV with all mutual funds
AMFI_SCHEME_URL = 'https://portal.amfiindia.com/DownloadSchemeData_Po.aspx?mf=0'
AMFI_SCHEME_LATEST_NAV_URL = 'https://www.amfiindia.com/spages/NAVAll.txt'

## Fetch Mutual Fund Scheme List

In [3]:
# Changing the column names to a more uniform and descriptive names
mf_col_names = [
    'AMC', 
    'Scheme_Code', 
    'Scheme_Name', 
    'Scheme_Type', 
    'Scheme_Category', 
    'Scheme_NAV_Name', 
    'Scheme_Min_Amt', 
    'Launch_Date', 
    'Closure_Date', 
    'ISIN_Div_Payout/Growth/Div_Reinvestment']

# --- Read Data ---
try:
    # Reading the URL, updating col names and datatypes
    mf_scheme_df = pd.read_csv(
        AMFI_SCHEME_URL, 
        names = mf_col_names, 
        # dtype = mf_scheme_dtypes, takes a dict of col:dtype 
        header = 0
    )
    print("📖 Successfully read data from AMFI Website!")
    # Row and column count 
    print(f"📋 Total Rows & Columns: {mf_scheme_df.shape}")
except (URLError, HTTPError) as e:
    # Exception to handle URL and HTTP errors
    print(f"🛜 Error reading CSV from URL: {e}.")
except Exception as e:
    # Exception to catch all other errors
    print(f"❌ Error occurred while reading CSV: {e}.")

📖 Successfully read data from AMFI Website!
📋 Total Rows & Columns: (16102, 10)


## Fetch Latest NAV Data From AMFI

In [4]:
# Changing the column names to a more uniform and descriptive names
nav_col_names = [
    'Scheme_Code',
    'ISIN_Div_Payout/Growth',
    'ISIN_Div_Reinvestment',
    'Scheme_Name', 
    'NAV', 
    'Latest_NAV_Date'
]

# --- Read Data ---
try:
    # Getting the latest NAV(Net Asset Value)
    latest_nav_df = pd.read_csv(
        AMFI_SCHEME_LATEST_NAV_URL,
        names = nav_col_names,
        header = 0, 
        sep = ';',    
    )
    print("📊 Successfully read latest NAV data from AMFI Website!")
    # Row and column count 
    print(f"📋 Total Rows & Columns: {latest_nav_df.shape}")
except (URLError, HTTPError) as e:
    # Exception to handle URL and HTTP errors
    print(f"🛜 Error reading CSV from URL: {e}.")
except Exception as e:
    # Exception to catch all other errors
    print(f"❌ Error occurred while reading TXT: {e}.")

# --- Sort & Filter Data ---
try:
    # Sort the values by scheme code in ascending order and updating the original df
    latest_nav_df.sort_values(
        'Scheme_Code', ascending = True, inplace = True)
    # Drop NaN, if Scheme name, nav and nav date are NaN 
    latest_nav_df.dropna(
        subset = ['Scheme_Name', 'NAV', 'Latest_NAV_Date'], 
        inplace = True)
    print("✅ Successfully removed NaN from latest_nav_df.")
    # Row and column count 
    print(f"🆕 Updated Rows & Columns: {latest_nav_df.shape}")
except Exception as e:
    # Exception to catch all other errors
    print(f"❌ Error occured while sorting scheme code/dropping NaN.")

📊 Successfully read latest NAV data from AMFI Website!
📋 Total Rows & Columns: (15048, 6)
✅ Successfully removed NaN from latest_nav_df.
🆕 Updated Rows & Columns: (13996, 6)


In [5]:
# Function to convert the dataframe column dtypes (not a dataframe copy)
# Need to import Pandas for this function to work!

print("⏳ Defining data type conversion function...")


def data_type_conversion (input_dict, input_df, df_name):
    """
    Converts dataframe columns to various (int, float, date/datetime, category and string) data types.

    Args:
        input_dict(dict): Key-Value pairs of 'col': 'dtype' to convert.
        input_df(dataframe): Dataframe you want the changes to be made.
        df_name(string): Pass the name of the Dataframe as a string for logs.

    Returns:
        Nothing returned, data type conversions are made to the passed dataframe(input_df)
    """

    print(f"🔀 Sarting data type conversion for the dataframe: {df_name}.")
    
    # --- Loop over the dict's items ---
    for col, dtype in input_dict.items():
        # --- Check if the column actually exists in the dataframe before changing data type ---
        if col in input_df.columns:
            print(f"  🚩 Attempting to convert '{col}' to type: '{dtype}'.")
            # Try-except for error handling
            try:
                # --- Apply conversion based on the dtype string from the dictionary ---
                if dtype == 'int':
                    # pd.to_numeric(errors='coerce') handles the dirty work of converting non-numeric garbage to NaN.
                    # .astype(dtype) comes after to enforce the specific target numeric type (float or the special Int64 that handles NaNs for integers) that you originally requested.
                    input_df[col] = pd.to_numeric(input_df[col], errors = 'coerce').astype('Int64')
                elif dtype == 'float':
                    # Coerce to numeric first, then ensure float type
                    input_df[col] = pd.to_numeric(input_df[col], errors = 'coerce').astype('float')
                elif dtype == 'date' or dtype == 'datetime':
                    input_df[col] = pd.to_datetime(input_df[col], errors = 'coerce')
                elif dtype == 'category':
                    input_df[col] = input_df[col].astype('category')
                elif dtype ==  'string':
                    input_df[col] = input_df[col].astype('string')
                else:
                    print(f"    🟨 Warning: Unknown dtype '{dtype}' specified for column '{col}'. Could not attempt conversion.")

                print(f"    ✅ Successfully converted '{col}' to ''{input_df[col].dtype}''")
            except Exception as e:
                print(f"  ❌ Error converting column '{col}' to ''{dtype}'', Error: {e}")
        else:
            print(f"🟨 Warning: '{col}' does not exists in '{df_name}' dataframe.")
    
    # --- Data conversion done ---
    print(f"✅ Finished data conversion for dataframe: {df_name}.")

print("✅ Defined data type conversion function.")

⏳ Defining data type conversion function...
✅ Defined data type conversion function.


## Process Data (Data Type Conversion)
1. Convert categorical values to 'category' dtype.
2. Convert dated columns to 'datetime' dtype.
3. Convert numerical columns to 'Int64' and 'float' based on the data.

In [6]:
# --- Data Type Conversion ---

# Converting to 'category' datatype will reduce the memory and increase the speed of queries on the dataframe
mf_scheme_dtypes = {
    'AMC': 'category',
    'Scheme_Code': 'int',
    'Scheme_Type': 'category',
    'Scheme_Category': 'category',
    'Launch_Date': 'datetime', 
    'Closure_Date': 'datetime'
}

# Call the function to convert the columns of mf_scheme_df
data_type_conversion(mf_scheme_dtypes, mf_scheme_df, 'mf_scheme_df')


# Dictionary of 'Col' : 'dtype' to convert the columns to a different dtype
nav_dtypes = {
    'Scheme_Code': 'int',
    'NAV': 'float',
    'Latest_NAV_Date': 'datetime'
}

# Call the function to convert the columns of latest_nav_df
data_type_conversion(nav_dtypes, latest_nav_df, 'latest_nav_df')

🔀 Sarting data type conversion for the dataframe: mf_scheme_df.
  🚩 Attempting to convert 'AMC' to type: 'category'.
    ✅ Successfully converted 'AMC' to ''category''
  🚩 Attempting to convert 'Scheme_Code' to type: 'int'.
    ✅ Successfully converted 'Scheme_Code' to ''Int64''
  🚩 Attempting to convert 'Scheme_Type' to type: 'category'.
    ✅ Successfully converted 'Scheme_Type' to ''category''
  🚩 Attempting to convert 'Scheme_Category' to type: 'category'.
    ✅ Successfully converted 'Scheme_Category' to ''category''
  🚩 Attempting to convert 'Launch_Date' to type: 'datetime'.
    ✅ Successfully converted 'Launch_Date' to ''datetime64[ns]''
  🚩 Attempting to convert 'Closure_Date' to type: 'datetime'.
    ✅ Successfully converted 'Closure_Date' to ''datetime64[ns]''
✅ Finished data conversion for dataframe: mf_scheme_df.
🔀 Sarting data type conversion for the dataframe: latest_nav_df.
  🚩 Attempting to convert 'Scheme_Code' to type: 'int'.
    ✅ Successfully converted 'Scheme_Code

In [7]:
merged_df = pd.merge(mf_scheme_df, latest_nav_df, on = 'Scheme_Code', how = 'inner')
merged_df

,AMC,Scheme_Code,Scheme_Name_x,Scheme_Type,Scheme_Category,Scheme_NAV_Name,Scheme_Min_Amt,Launch_Date,Closure_Date,ISIN_Div_Payout/Growth/Div_Reinvestment,ISIN_Div_Payout/Growth,ISIN_Div_Reinvestment,Scheme_Name_y,NAV,Latest_NAV_Date
0,Aditya Birla Sun Life AMC Limited,100033,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,5000,1995-02-24,1995-02-24,INF209K01165,INF209K01165,-,Aditya Birla Sun Life Equity Advantage Fund - ...,841.0900,2025-05-02
1,Aditya Birla Sun Life AMC Limited,100034,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -R...,5000,1995-02-24,1995-02-24,INF209K01157INF209K01CE5,INF209K01157,INF209K01CE5,Aditya Birla Sun Life Equity Advantage Fund -R...,130.4500,2025-05-02
2,Aditya Birla Sun Life AMC Limited,119433,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -D...,5000,1995-02-24,1995-02-24,INF209K01VG0INF209K01VI6,INF209K01VG0,INF209K01VI6,Aditya Birla Sun Life Equity Advantage Fund -D...,200.1300,2025-05-02
3,Aditya Birla Sun Life AMC Limited,119436,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,5000,1995-02-24,1995-02-24,INF209K01VH8,INF209K01VH8,-,Aditya Birla Sun Life Equity Advantage Fund - ...,933.8600,2025-05-02
4,Aditya Birla Sun Life AMC Limited,100037,Aditya Birla Sun Life Income Fund,Open Ended,Debt Scheme - Medium to Long Duration Fund,Aditya Birla Sun Life Income Fund - Regular - ...,5000,1994-01-01,1994-01-01,INF209K01587INF209K01CY3,INF209K01587,INF209K01CY3,Aditya Birla Sun Life Income Fund - Regular - ...,13.5901,2025-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13991,Groww Asset Management Limited,153456,Groww Nifty 500 Momentum 50 ETF FOF,Open Ended,Other Scheme - FoF Domestic,Groww Nifty 500 Momentum 50 ETF FOF - Regular ...,Rs. 500 and in multiples of Re. 1 thereafter,2025-04-03,2025-04-17,INF666M01KC9,INF666M01KC9,-,Groww Nifty 500 Momentum 50 ETF FOF - Regular ...,9.8682,2025-05-02
13992,Kotak Mahindra Asset Management Company Limited.,153461,Kotak Nifty Top 10 Equal Weight Index Fund,Open Ended,Other Scheme - Index Funds,Kotak Nifty Top 10 Equal Weight Index Fund-Dir...,Rs.100/- and any amount thereafter,2025-04-07,NaT,INF174KA1WD0,INF174KA1WD0,-,Kotak Nifty Top 10 Equal Weight Index Fund-Dir...,10.0490,2025-05-02
13993,Kotak Mahindra Asset Management Company Limited.,153462,Kotak Nifty Top 10 Equal Weight Index Fund,Open Ended,Other Scheme - Index Funds,Kotak Nifty Top 10 Equal Weight Index Fund-Dir...,Rs.100/- and any amount thereafter,2025-04-07,NaT,INF174KA1WE8INF174KA1WF5,INF174KA1WE8,INF174KA1WF5,Kotak Nifty Top 10 Equal Weight Index Fund-Dir...,10.0490,2025-05-02
13994,Kotak Mahindra Asset Management Company Limited.,153463,Kotak Nifty Top 10 Equal Weight Index Fund,Open Ended,Other Scheme - Index Funds,Kotak Nifty Top 10 Equal Weight Index Fund-Reg...,Rs.100/- and any amount thereafter,2025-04-07,NaT,INF174KA1WG3,INF174KA1WG3,-,Kotak Nifty Top 10 Equal Weight Index Fund-Reg...,10.0480,2025-05-02


In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13996 entries, 0 to 13995
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   AMC                                      13996 non-null  category      
 1   Scheme_Code                              13996 non-null  Int64         
 2   Scheme_Name_x                            13996 non-null  object        
 3   Scheme_Type                              13996 non-null  category      
 4   Scheme_Category                          13996 non-null  category      
 5   Scheme_NAV_Name                          13996 non-null  object        
 6   Scheme_Min_Amt                           13987 non-null  object        
 7   Launch_Date                              13995 non-null  datetime64[ns]
 8   Closure_Date                             3698 non-null   datetime64[ns]
 9   ISIN_Div_Payout/Growth/Div_Reinvestment